In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import cifar10,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt

import sys
import os

import numpy as np

class COGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/02/18/coupled_gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.d1, self.d2 = self.build_discriminators()
        self.d1.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d2.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.g1, self.g2 = self.build_generators()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img1 = self.g1(z)
        img2 = self.g2(z)

        # For the combined model we will only train the generators
        self.d1.trainable = False
        self.d2.trainable = False

        # The valid takes generated images as input and determines validity
        valid1 = self.d1(img1)
        valid2 = self.d2(img2)

        # The combined model  (stacked generators and discriminators)
        # Trains generators to fool discriminators
        self.combined = Model(z, [valid1, valid2])
        self.combined.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
                                    optimizer=optimizer)

    def build_generators(self):
        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img1 = model(noise)
        img2 = model(noise)



        return Model(noise, img1), Model(noise, img2)

    def build_discriminators(self):
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img1 = Input(shape=self.img_shape)
        img2 = Input(shape=self.img_shape)
        validity1 = model(img1)
        validity2 = model(img2)

        return Model(img1, validity1), Model(img2, validity2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Images in domain A and B (rotated)
        X1 = X_train[:int(X_train.shape[0]/2)]
        X2 = X_train[int(X_train.shape[0]/2):]
        X2 = scipy.ndimage.interpolation.rotate(X2, 90, axes=(1, 2))

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X1.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                # imgs = X_train[index * batch_size:(index + 1) * batch_size]

                imgs1 = X1[index * batch_size:(index + 1) * batch_size]
                imgs2 = X2[index * batch_size:(index + 1) * batch_size]
                # print(len(imgs1))

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a batch of new images
                gen_imgs1 = self.g1.predict(noise)
                gen_imgs2 = self.g2.predict(noise)

                # Train the discriminators
                d1_loss_real = self.d1.train_on_batch(imgs1, valid)
                d2_loss_real = self.d2.train_on_batch(imgs2, valid)
                d1_loss_fake = self.d1.train_on_batch(gen_imgs1, fake)
                d2_loss_fake = self.d2.train_on_batch(gen_imgs2, fake)
                d1_loss = 0.5 * np.add(d1_loss_real, d1_loss_fake)
                d2_loss = 0.5 * np.add(d2_loss_real, d2_loss_fake)

                # ------------------
                #  Train Generators
                # ------------------

                g_loss = self.combined.train_on_batch(noise, [valid, valid])

                # Plot the progress
                print("epoch:%d step:%d [D1 loss: %f, acc.: %.2f%%] [D2 loss: %f, acc.: %.2f%%] [G loss: %f]" \
                      % (epoch, global_step, d1_loss[0], 100 * d1_loss[1], d2_loss[0], 100 * d2_loss[1], g_loss[0]))

                # If at save interval => save generated image samples
                if  global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * int(c/2), 100))
        gen_imgs1 = self.g1.predict(noise)
        gen_imgs2 = self.g2.predict(noise)

        generated_images = np.concatenate([gen_imgs1, gen_imgs2])

        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(generated_images[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        if not os.path.isdir('images_cogan_fashion_mnist'):
            os.mkdir('images_cogan_fashion_mnist')
        fig.savefig("images_cogan_fashion_mnist/epoch_%d_step_%d.png" % (epoch, global_step))
        plt.close()


if __name__ == '__main__':
    gan = COGAN()
    gan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D1 loss: 1.470701, acc.: 21.09%] [D2 loss: 0.816556, acc.: 54.69%] [G loss: 1.557644]
epoch:0 step:2 [D1 loss: 0.579873, acc.: 69.53%] [D2 loss: 0.394010, acc.: 81.25%] [G loss: 3.853537]
epoch:0 step:3 [D1 loss: 0.397031, acc.: 81.25%] [D2 loss: 0.282194, acc.: 90.62%] [G loss: 2.766824]
epoch:0 step:4 [D1 loss: 0.216100, acc.: 93.75%] [D2 loss: 0.155508, acc.: 97.66%] [G loss: 1.041335]
epoch:0 step:5 [D1 loss: 0.108126, acc.: 97.66%] [D2 loss: 0.054969, acc.: 100.00%] [G loss: 0.434176]
epoch:0 step:6 [D1 loss: 0.086661, acc.: 97.66%] [D2 loss: 0.042683, acc.: 99.22%] [G loss: 0.347059]
epoch:0 step:7 [D1 loss: 0.081104, acc.: 98.44%] [D2 loss: 0.040993, acc.: 100.00%] [G loss: 0.417302]
epoch:0 step:8 [D1 loss: 0.529722, acc.: 76.56%] [D2 loss: 0.065822, acc.: 98.44%] [G loss: 1.418308]
epoch:0 step:9 [D1 loss: 2.247030, acc.: 40.62%] [D2 loss: 0.538067, acc.: 71.09%] [G loss: 6.411439]
epoch:0 step:10 [D1 loss: 1.927163, acc.: 16.41%] [D2 loss: 0.721296, acc.: 66.4

epoch:0 step:82 [D1 loss: 0.679056, acc.: 66.41%] [D2 loss: 0.189454, acc.: 94.53%] [G loss: 1.745120]
epoch:0 step:83 [D1 loss: 0.896270, acc.: 62.50%] [D2 loss: 0.476964, acc.: 77.34%] [G loss: 4.482375]
epoch:0 step:84 [D1 loss: 0.983148, acc.: 37.50%] [D2 loss: 0.765778, acc.: 54.69%] [G loss: 4.565224]
epoch:0 step:85 [D1 loss: 1.122394, acc.: 30.47%] [D2 loss: 0.647654, acc.: 64.06%] [G loss: 4.004610]
epoch:0 step:86 [D1 loss: 0.765280, acc.: 62.50%] [D2 loss: 0.459010, acc.: 78.91%] [G loss: 4.259963]
epoch:0 step:87 [D1 loss: 0.981216, acc.: 42.97%] [D2 loss: 0.599294, acc.: 67.97%] [G loss: 4.000191]
epoch:0 step:88 [D1 loss: 0.548740, acc.: 68.75%] [D2 loss: 0.394038, acc.: 75.78%] [G loss: 1.359823]
epoch:0 step:89 [D1 loss: 0.488722, acc.: 75.78%] [D2 loss: 0.241139, acc.: 92.19%] [G loss: 1.443160]
epoch:0 step:90 [D1 loss: 0.565516, acc.: 71.88%] [D2 loss: 0.222887, acc.: 96.09%] [G loss: 3.099092]
epoch:0 step:91 [D1 loss: 0.852505, acc.: 48.44%] [D2 loss: 0.400161, acc

...

epoch:49 step:23073 [D1 loss: 0.269513, acc.: 89.06%] [D2 loss: 0.453408, acc.: 82.81%] [G loss: 4.404732]
epoch:49 step:23074 [D1 loss: 0.251189, acc.: 89.84%] [D2 loss: 0.472217, acc.: 79.69%] [G loss: 4.507648]
epoch:49 step:23075 [D1 loss: 0.340277, acc.: 88.28%] [D2 loss: 0.374652, acc.: 83.59%] [G loss: 4.786761]
epoch:49 step:23076 [D1 loss: 0.347871, acc.: 84.38%] [D2 loss: 0.594732, acc.: 72.66%] [G loss: 3.712701]
epoch:49 step:23077 [D1 loss: 0.319576, acc.: 84.38%] [D2 loss: 0.623690, acc.: 73.44%] [G loss: 3.646394]
epoch:49 step:23078 [D1 loss: 0.300414, acc.: 89.84%] [D2 loss: 0.489386, acc.: 74.22%] [G loss: 4.856203]
epoch:49 step:23079 [D1 loss: 0.236308, acc.: 92.19%] [D2 loss: 0.547690, acc.: 68.75%] [G loss: 3.212050]
epoch:49 step:23080 [D1 loss: 0.407070, acc.: 79.69%] [D2 loss: 0.772914, acc.: 67.19%] [G loss: 3.044438]
epoch:49 step:23081 [D1 loss: 0.261190, acc.: 90.62%] [D2 loss: 0.571086, acc.: 68.75%] [G loss: 3.612420]
epoch:49 step:23082 [D1 loss: 0.39521

epoch:49 step:23152 [D1 loss: 0.205207, acc.: 92.19%] [D2 loss: 0.890761, acc.: 56.25%] [G loss: 4.361725]
epoch:49 step:23153 [D1 loss: 0.315782, acc.: 86.72%] [D2 loss: 0.675611, acc.: 69.53%] [G loss: 3.157538]
epoch:49 step:23154 [D1 loss: 0.207201, acc.: 90.62%] [D2 loss: 1.051677, acc.: 50.00%] [G loss: 3.872147]
epoch:49 step:23155 [D1 loss: 0.399507, acc.: 76.56%] [D2 loss: 0.775502, acc.: 61.72%] [G loss: 3.551152]
epoch:49 step:23156 [D1 loss: 0.201256, acc.: 90.62%] [D2 loss: 0.992010, acc.: 53.91%] [G loss: 4.187747]
epoch:49 step:23157 [D1 loss: 0.187942, acc.: 92.19%] [D2 loss: 1.156546, acc.: 50.00%] [G loss: 4.507476]
epoch:49 step:23158 [D1 loss: 0.284741, acc.: 85.16%] [D2 loss: 1.162104, acc.: 46.88%] [G loss: 3.255235]
epoch:49 step:23159 [D1 loss: 0.296517, acc.: 84.38%] [D2 loss: 0.936399, acc.: 54.69%] [G loss: 3.859306]
epoch:49 step:23160 [D1 loss: 0.304156, acc.: 85.16%] [D2 loss: 1.127237, acc.: 53.12%] [G loss: 3.019516]
epoch:49 step:23161 [D1 loss: 0.26719

epoch:49 step:23231 [D1 loss: 0.251274, acc.: 89.06%] [D2 loss: 0.820437, acc.: 58.59%] [G loss: 3.511186]
epoch:49 step:23232 [D1 loss: 0.275613, acc.: 85.94%] [D2 loss: 1.164948, acc.: 49.22%] [G loss: 3.054584]
epoch:49 step:23233 [D1 loss: 0.248935, acc.: 86.72%] [D2 loss: 1.012741, acc.: 56.25%] [G loss: 3.407205]
epoch:49 step:23234 [D1 loss: 0.198658, acc.: 92.19%] [D2 loss: 1.010474, acc.: 59.38%] [G loss: 4.234097]
epoch:49 step:23235 [D1 loss: 0.366810, acc.: 78.91%] [D2 loss: 1.028855, acc.: 53.91%] [G loss: 2.964586]
epoch:49 step:23236 [D1 loss: 0.321738, acc.: 86.72%] [D2 loss: 0.792414, acc.: 59.38%] [G loss: 3.538996]
epoch:49 step:23237 [D1 loss: 0.273829, acc.: 85.16%] [D2 loss: 0.718059, acc.: 64.06%] [G loss: 3.191454]
epoch:49 step:23238 [D1 loss: 0.327686, acc.: 86.72%] [D2 loss: 1.127622, acc.: 51.56%] [G loss: 3.403191]
epoch:49 step:23239 [D1 loss: 0.229042, acc.: 89.06%] [D2 loss: 1.017362, acc.: 54.69%] [G loss: 3.278674]
epoch:49 step:23240 [D1 loss: 0.26316

epoch:49 step:23310 [D1 loss: 0.225254, acc.: 89.06%] [D2 loss: 0.913898, acc.: 58.59%] [G loss: 4.322320]
epoch:49 step:23311 [D1 loss: 0.218813, acc.: 91.41%] [D2 loss: 0.896547, acc.: 61.72%] [G loss: 4.084677]
epoch:49 step:23312 [D1 loss: 0.258628, acc.: 90.62%] [D2 loss: 1.128532, acc.: 50.00%] [G loss: 4.269853]
epoch:49 step:23313 [D1 loss: 0.299219, acc.: 86.72%] [D2 loss: 0.776692, acc.: 59.38%] [G loss: 3.980671]
epoch:49 step:23314 [D1 loss: 0.275092, acc.: 88.28%] [D2 loss: 1.040524, acc.: 50.78%] [G loss: 3.717147]
epoch:49 step:23315 [D1 loss: 0.160568, acc.: 94.53%] [D2 loss: 1.079505, acc.: 53.91%] [G loss: 3.629598]
epoch:49 step:23316 [D1 loss: 0.193757, acc.: 89.84%] [D2 loss: 0.817955, acc.: 58.59%] [G loss: 2.794924]
epoch:49 step:23317 [D1 loss: 0.246083, acc.: 88.28%] [D2 loss: 0.990881, acc.: 56.25%] [G loss: 3.663710]
epoch:49 step:23318 [D1 loss: 0.363839, acc.: 82.03%] [D2 loss: 1.023386, acc.: 51.56%] [G loss: 3.874149]
epoch:49 step:23319 [D1 loss: 0.32945

epoch:49 step:23388 [D1 loss: 0.226238, acc.: 90.62%] [D2 loss: 1.030589, acc.: 48.44%] [G loss: 2.672251]
epoch:49 step:23389 [D1 loss: 0.287139, acc.: 85.94%] [D2 loss: 1.048222, acc.: 49.22%] [G loss: 3.438441]
epoch:49 step:23390 [D1 loss: 0.248242, acc.: 85.94%] [D2 loss: 0.927639, acc.: 52.34%] [G loss: 3.990171]
epoch:49 step:23391 [D1 loss: 0.289256, acc.: 86.72%] [D2 loss: 0.866781, acc.: 57.03%] [G loss: 3.634892]
epoch:49 step:23392 [D1 loss: 0.258985, acc.: 88.28%] [D2 loss: 0.871066, acc.: 58.59%] [G loss: 3.489176]
epoch:49 step:23393 [D1 loss: 0.458205, acc.: 76.56%] [D2 loss: 1.173250, acc.: 37.50%] [G loss: 3.628700]
epoch:49 step:23394 [D1 loss: 0.194683, acc.: 91.41%] [D2 loss: 0.942932, acc.: 54.69%] [G loss: 4.080091]
epoch:49 step:23395 [D1 loss: 0.201412, acc.: 92.19%] [D2 loss: 0.897708, acc.: 57.81%] [G loss: 3.669916]
epoch:49 step:23396 [D1 loss: 0.224638, acc.: 92.97%] [D2 loss: 0.777378, acc.: 60.16%] [G loss: 3.622508]
epoch:49 step:23397 [D1 loss: 0.19620